In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Load the CIFAR-10 dataset
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()


170498071/170498071 [==============================] - 7s 0us/step


In [ ]:
# Normalize pixel values to [0,1]
train_images = train_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0

In [ ]:
# Define the MobileNetV2 model with pre-trained weights
base_model = tf.keras.applications.MobileNetV2(input_shape=(32, 32, 3),
                                               include_top=False,
                                               weights='imagenet')

In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base layers of the model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
# Define data generators for training and validation
train_datagen = ImageDataGenerator(rotation_range=20,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   horizontal_flip=True,
                                   validation_split=0.2)
train_generator = train_datagen.flow(train_images, train_labels, batch_size=32, subset='training')
valid_generator = train_datagen.flow(train_images, train_labels, batch_size=32, subset='validation')


In [ ]:
# Train the model
model.fit(train_generator,
          epochs=10,
          validation_data=valid_generator)

Epoch 1/10
1250/1250 [==============================] - 100s 76ms/step - loss: 2.0040 - accuracy: 0.1056 - val_loss: 1.9506 - val_accuracy: 0.1374
Epoch 2/10
1250/1250 [==============================] - 94s 75ms/step - loss: 1.9249 - accuracy: 0.1196 - val_loss: 1.9198 - val_accuracy: 0.0790
Epoch 3/10
1250/1250 [==============================] - 91s 73ms/step - loss: 1.9041 - accuracy: 0.1191 - val_loss: 1.9008 - val_accuracy: 0.1285
Epoch 4/10
1250/1250 [==============================] - 94s 75ms/step - loss: 1.8869 - accuracy: 0.1171 - val_loss: 1.8876 - val_accuracy: 0.1528
Epoch 5/10
 396/1250 [========>.....................] - ETA: 49s - loss: 1.8789 - accuracy: 0.1174

In [10]:
model1.save('my_model')

In [11]:
# Convert the model to TFLite format
converter = tf.lite.TFLiteConverter.from_saved_model('my_model')
tflite_model = converter.convert()

# Save the TFLite model
with open('mobilenetmodel.tflite', 'wb') as f:
    f.write(tflite_model)

In [1]:
# Importing Packages
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import os

from keras.preprocessing import image
from keras.models import Model
from keras.applications import imagenet_utils
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications.mobilenet import MobileNet

import tensorflow.keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from keras import optimizers
from keras.layers import Resizing
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten,Conv2D, MaxPooling2D,BatchNormalization,LayerNormalization

In [2]:
# Loading the Dataset and getting size of it
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
print("Size of the train images", x_train.shape)
print("Size of the test images", x_test.shape)

170498071/170498071 [==============================] - 4s 0us/step
Size of the train images (50000, 32, 32, 3)
Size of the test images (10000, 32, 32, 3)


In [3]:
x_test, y_test = x_test[:5000], y_test[:5000]
x_val, y_val = x_train[:5000], y_train[:5000]
x_train, y_train = x_train[30000:], y_train[30000:]

In [4]:
print("Training data size: ", x_train.shape)
print("Validation data size: ", x_val.shape)
print("Test data size: ", x_test.shape)
print("Training data Labels", y_train.shape)
print("Validation data Labels", y_val.shape)

Training data size:  (20000, 32, 32, 3)
Validation data size:  (5000, 32, 32, 3)
Test data size:  (5000, 32, 32, 3)
Training data Labels (20000, 1)
Validation data Labels (5000, 1)


In [5]:
from keras.utils import np_utils

num_classes = 10

# Convert class vectors to binary class matrices.
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)
y_val = np_utils.to_categorical(y_val, num_classes)

In [6]:
from sklearn.utils import shuffle

x_val, y_val  = shuffle(x_val, y_val)
x_train, y_train = shuffle(x_train, y_train)

In [7]:
# Base model for the mobilenet model
mobnet_model=MobileNet(weights='imagenet',include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.

17225924/17225924 [==============================] - 0s 0us/step


In [8]:
image_size = (224,224)

from keras import layers
from keras import models

num_classes = 10

model1 = Sequential()
keras.layers.experimental.preprocessing.Resizing(image_size[0], image_size[1], interpolation="bilinear", input_shape=x_train.shape[1:]),
           
model1.add(mobnet_model) 
model1.add(GlobalAveragePooling2D())

model1.add(Dense(1024,activation=('relu')))
model1.add(Dense(512,activation=('relu'))) 
model1.add(Dense(256,activation=('relu'))) 
model1.add(Dropout(0.5))
model1.add(Dense(128,activation=('relu')))
model1.add(Dropout(0.5))
model1.add(Dense(10,activation=('softmax')))

model1.compile(loss='categorical_crossentropy',
              optimizer= 'Adam',
              metrics=['acc'])
# Creating the model and compiling it
model1.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
# a summary of the model
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenet_1.00_224 (Functio  (None, None, None, 1024)  3228864  
 nal)                                                            
                                                                 
 global_average_pooling2d (G  (None, 1024)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 1024)              1049600   
                                                                 
 dense_1 (Dense)             (None, 512)               524800    
                                                                 
 dense_2 (Dense)             (None, 256)               131328    
                                                                 
 dropout (Dropout)           (None, 256)               0

In [9]:
# training the model for 20 epochs
history = model1.fit(x_train, y_train, batch_size = 64, epochs = 15, validation_data = (x_test, y_test))

Epoch 1/15
313/313 [==============================] - 234s 716ms/step - loss: 1.6647 - accuracy: 0.4336 - val_loss: 1.2052 - val_accuracy: 0.6114
Epoch 2/15
313/313 [==============================] - 229s 732ms/step - loss: 1.1536 - accuracy: 0.6439 - val_loss: 1.1817 - val_accuracy: 0.6246
Epoch 3/15
313/313 [==============================] - 221s 708ms/step - loss: 0.9774 - accuracy: 0.7113 - val_loss: 0.9866 - val_accuracy: 0.6960
Epoch 4/15
313/313 [==============================] - 222s 708ms/step - loss: 0.8680 - accuracy: 0.7448 - val_loss: 1.0418 - val_accuracy: 0.6726
Epoch 5/15
313/313 [==============================] - 220s 702ms/step - loss: 0.7660 - accuracy: 0.7716 - val_loss: 0.9148 - val_accuracy: 0.7164
Epoch 6/15
313/313 [==============================] - 219s 700ms/step - loss: 0.7217 - accuracy: 0.7872 - val_loss: 0.9830 - val_accuracy: 0.7204
Epoch 7/15
313/313 [==============================] - 220s 704ms/step - loss: 0.6640 - accuracy: 0.8031 - val_loss: 0.8423 -

In [13]:
print(model1.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenet_1.00_224 (Functio  (None, None, None, 1024)  3228864  
 nal)                                                            
                                                                 
 global_average_pooling2d (G  (None, 1024)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 1024)              1049600   
                                                                 
 dense_1 (Dense)             (None, 512)               524800    
                                                                 
 dense_2 (Dense)             (None, 256)               131328    
                                                                 
 dropout (Dropout)           (None, 256)               0